In [360]:
import os, sys
from utils import *
from download_DB import download_db
import glob
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

from datetime import date
hoje =  pd.to_datetime(date.today())

In [361]:
files = []
for file in glob.glob("./Queries/v*.txt",):
    a,b = os.path.split(file)
    files.append(b.split(".")[0])
directory = download_db(lista_download=files)


******************** Baixando arquivos *********************

Carregando arquivo de log...
vmonitoramentoleilao já foi baixado hoje: Dia: 17/01/23 - Horário: 14:30:36.  Portanto não foi baixado novamente.
vmonitoramentoug já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:06.  Portanto não foi baixado novamente.
vmonitoramentousina já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:10.  Portanto não foi baixado novamente.
vrapeelacesso já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:22.  Portanto não foi baixado novamente.
vrapeelcontratorecurso já foi baixado hoje: Dia: 17/01/23 - Horário: 14:35:46.  Portanto não foi baixado novamente.
vrapeelcronograma já foi baixado hoje: Dia: 17/01/23 - Horário: 14:36:43.  Portanto não foi baixado novamente.
vrapeelempreendimento já foi baixado hoje: Dia: 17/01/23 - Horário: 14:36:54.  Portanto não foi baixado novamente.
vrapeellicenciamento já foi baixado hoje: Dia: 17/01/23 - Horário: 14:37:38.  Portanto não foi baixado novamente.
vrapeelo

In [362]:
# Lista de colunas que serão usadas várias vezes para a realização de merges entre os dataframes
list_id_data = ['DthEnvio','IdeUsinaOutorga']
list_id_ug = ['IdeUsinaOutorga','NumOperacaoUg']

# Informações de Usinas

## Carrega banco de dados

In [363]:
cols = ["IdeUsinaOutorga","DatCanteiroObraRealizado","DatDesvioRioRealizado","DatEnchimentoRealizado","DatConclusaoSisTransRealizado","DatPrevisaoIniciobra","IdcObraParalisada","IdcUsinaMonitorada","DatMontagemOutorgado","DatConcretagemRealizado","DatConclusaoTorresRealizado","DatInicioObraOutorgado","DatConcretagemOutorgado","DatSisTransmissaoRealizado","IdcSituacaoObra","IdcSemPrevisao","DatComissionamentoUGRealizado","DatMonitoramento","SigTipoGeracao","DscComercializacaoEnergia","DatInicioObraRealizado","DatMontagemRealizado"]
df_usina = pd.read_parquet(os.path.join(directory,"vmonitoramentousina.gzip"))[cols]
df_usina = df_usina[df_usina.IdcUsinaMonitorada == "Sim"].reset_index(drop=True)

In [364]:
monitoramentoleilao  = pd.read_parquet(os.path.join(directory,"vmonitoramentoleilao.gzip"))[['IdeUsinaOutorga','CodLeilao', 'DatInicioSuprimento']]
monitoramentoleilao.dropna(subset='DatInicioSuprimento',inplace=True)
monitoramentoleilao = monitoramentoleilao.loc[monitoramentoleilao.groupby('IdeUsinaOutorga').DatInicioSuprimento.idxmin()]
df_usina = pd.merge(df_usina,monitoramentoleilao,how="left",on="IdeUsinaOutorga")

In [365]:
# Carrega BD vrapeelcronograma => rapeel_cronograma
cols = ['IdeUsinaOutorga','CodCeg','DthEnvio','DatRealizacaoII','DatPrevistaAprovacaoIII','DatRealizacaoIII','DatRealizacaoIX','DatRealizacaoXI','DatRealizacaoXIII','DatRealizacaoXIV','DatRealizacaoXII','DatRealizacaoVII','DatRealizacaoVIII','DatRealizacaoVI','DatRealizacaoX','DatRealizacaoV','DatRealizacaoIV']
rename_cols = {
        'DatRealizacaoII' : 'canteiroReal',
        'DatPrevistaAprovacaoIII' : 'prev_IO',
        'DatRealizacaoIII' : 'IO_real',
        'DatRealizacaoIX' : 'DesvRio_real',
        'DatRealizacaoXI' : 'Ench_Real',
        'DatRealizacaoXIII' : 'IOTrans_Real',
        'DatRealizacaoXIV' : 'Conc_Trans_REAL',
        'DatRealizacaoXII' : 'Comiss_Real',
        'DatRealizacaoVII' : 'ME_Real_conc_eol',
        'DatRealizacaoVIII' : 'Conc_Combust_Real'}
        
rapeel_cronograma = pd.read_parquet(os.path.join(directory,"vrapeelcronograma.gzip"))[cols].rename(columns=rename_cols)
rapeel_cronograma = rapeel_cronograma.loc[rapeel_cronograma.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
df_usina = pd.merge(df_usina,rapeel_cronograma,how="left",on="IdeUsinaOutorga")

In [366]:
# Carrega BD vrapeelcontratorecurso => mercado
cols = ['DthEnvio','IdeUsinaOutorga','DatConclusaoIII',]
rename_cols = {'DatConclusaoIII':'DatConclusaoACL'}
mercado = pd.read_parquet(os.path.join(directory,"vrapeelcontratorecurso.gzip"))[cols].rename(columns=rename_cols)
mercado = mercado.loc[mercado.groupby("IdeUsinaOutorga").DthEnvio.idxmax()]
del mercado['DthEnvio']
df_usina = pd.merge(df_usina,mercado,on="IdeUsinaOutorga",how="left")

In [367]:
# Carrega BD vrapeellicenciamento => ambiental_datas
cols = ["DatValidadeIV","DatValidadeI","DthEnvio","IdeUsinaOutorga","DatValidadeV"]
rename_cols = {'DatValidadeIV': 'DatValidadeLI','DatValidadeI': 'DatValidadeLP','DatValidadeV': 'DatValidadeLO'}
ambiental_datas = pd.read_parquet(os.path.join(directory,"vrapeellicenciamento.gzip"))[cols].rename(columns=rename_cols)
ambiental_datas = ambiental_datas.loc[ambiental_datas.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del ambiental_datas['DthEnvio']
df_usina = pd.merge(df_usina,ambiental_datas,on="IdeUsinaOutorga",how="left")

In [368]:
# Carrega BD vrapeelacesso => acesso_contratos
cols = ["DthEnvio","NomSitContratoI","NomSitContratoIV","NomSituacaoContratoIII","NomSitContratoII","IdeUsinaOutorga"]
rename_cols = {'NomSitContratoI': 'NomSitContratoCCD','NomSitContratoIV': 'NomSitContratoCUST','NomSituacaoContratoIII': 'NomSituacaoContratoCUSD','NomSitContratoII': 'NomSitContratoCCT'}
acesso_contratos = pd.read_parquet(os.path.join(directory,"vrapeelacesso.gzip"))[cols].rename(columns=rename_cols)
acesso_contratos = acesso_contratos.loc[acesso_contratos.groupby('IdeUsinaOutorga').DthEnvio.idxmax()]
del acesso_contratos['DthEnvio']
df_usina = pd.merge(df_usina,acesso_contratos,on="IdeUsinaOutorga",how="left")

##  Cria colunas com definições e regras

In [369]:
dict_validades = {
    'DatValidadeLO' : 'LO',
    'DatValidadeLI' : 'LI',
    'DatValidadeLP' : "LP"
}

df_usina['ValidadeAmbiental'] = pd.NA
for validade in dict_validades:
    df_usina.loc[df_usina.ValidadeAmbiental.isna(),'ValidadeAmbiental'] = df_usina[validade]
df_usina[['IdeUsinaOutorga'] +list(dict_validades.keys()) + ['ValidadeAmbiental']].sample(5)

,IdeUsinaOutorga,DatValidadeLO,DatValidadeLI,DatValidadeLP,ValidadeAmbiental
1310,48574,2031-07-12,2031-07-12,2031-07-12,2031-07-12
559,38087,2027-08-22,2025-12-22,2021-05-11,2027-08-22
935,46090,2030-08-06,2030-08-06,2030-08-06,2030-08-06
1232,47401,2031-10-05,2031-10-05,2031-10-05,2031-10-05
1126,47249,NaT,NaT,2031-03-03,2031-03-03


In [370]:
#WORK.dadosAcesso t1.'DthEnvio' LABEL="datadainformacao" AS datadainformacao

df_usina['CondicaoAmbiental'] = pd.NA

for validade in reversed(dict_validades):
    df_usina.loc[df_usina[validade].notna(),'CondicaoAmbiental'] = dict_validades[validade]

df_usina.loc[(df_usina.CondicaoAmbiental.isna()) & (df_usina.DthEnvio.notna()),'CondicaoAmbiental'] = 'Sem LP'
df_usina.loc[(df_usina.CondicaoAmbiental.isna()),'CondicaoAmbiental'] = 'Não Informado'


df_usina[list(dict_validades) + ['ValidadeAmbiental','CondicaoAmbiental']].sample(10)

display(df_usina[list(dict_validades.keys()) + ['CondicaoAmbiental','ValidadeAmbiental']])

df_usina['CondicaoConexao'] = "Não informado"

nsa = "Não se Aplica"
nass = "Não Assinado"
valido = "Válido"
vencido = "Vencido"

df_usina.loc[
    ((df_usina.NomSitContratoCCD ==  nsa)&
    (df_usina.NomSitContratoCCT == nsa) &
    df_usina.DthEnvio.notna()
    )
    ,'CondicaoConexao'] = "Verificar"

df_usina.loc[
(    ((df_usina.NomSitContratoCCD ==  valido) |
    (df_usina.NomSitContratoCCT == valido)) &
    df_usina.DthEnvio.notna())
    
    ,'CondicaoConexao'] = "OK"

df_usina.loc[
    (((df_usina.NomSitContratoCCD ==  vencido) |
    (df_usina.NomSitContratoCCT == vencido) |
    (df_usina.NomSitContratoCCD == nass) |
    (df_usina.NomSitContratoCCT == nass )) &
    df_usina.DthEnvio.notna())
    ,'CondicaoConexao'] = "Sem Conexão"
display(df_usina[['IdeUsinaOutorga'] + ['NomSitContratoCCD','NomSitContratoCCT','DthEnvio','CondicaoConexao']].sample(10))


df_usina['CondicaoUso'] = "Não informado"

df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  nsa)&
    (df_usina.NomSitContratoCUST == nsa) 
    )
    ,'CondicaoUso'] = "Verificar"


df_usina.loc[
    ((df_usina.NomSituacaoContratoCUSD ==  valido) |
    (df_usina.NomSitContratoCUST == valido) 
    )
    ,'CondicaoUso'] = "OK"

df_usina.loc[
    (((df_usina.NomSituacaoContratoCUSD ==  vencido) |
    (df_usina.NomSitContratoCUST == vencido) |
    (df_usina.NomSituacaoContratoCUSD == nass) |
    (df_usina.NomSitContratoCUST == nass )))
    ,'CondicaoUso'] = "Sem Uso"

display(df_usina[['IdeUsinaOutorga'] + ['NomSituacaoContratoCUSD','NomSitContratoCUST','CondicaoConexao']].sample(10))


df_usina["PPA"] = "Ambos"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "ACR"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.isna())),
    "PPA"] = "Nenhum"

df_usina.loc[
    (((df_usina.DscComercializacaoEnergia == "Fora do ACR") & 
        df_usina.DatConclusaoACL.notna())),
    "PPA"] = "ACL"

display(df_usina[['IdeUsinaOutorga'] + ['DscComercializacaoEnergia','DatConclusaoACL','PPA']].sample(10))

,DatValidadeLO,DatValidadeLI,DatValidadeLP,CondicaoAmbiental,ValidadeAmbiental
0,NaT,2026-03-09,2002-12-11,LI,2026-03-09
1,NaT,NaT,NaT,Sem LP,NaT
2,2026-02-22,2028-06-28,2000-03-29,LO,2026-02-22
3,NaT,NaT,NaT,Não Informado,NaT
4,NaT,2009-03-10,1999-01-09,LI,2009-03-10
...,...,...,...,...,...
2441,NaT,NaT,NaT,Não Informado,NaT
2442,NaT,NaT,NaT,Não Informado,NaT
2443,NaT,NaT,NaT,Não Informado,NaT
2444,NaT,NaT,NaT,Não Informado,NaT


,IdeUsinaOutorga,NomSitContratoCCD,NomSitContratoCCT,DthEnvio,CondicaoConexao
1262,47454,Não se Aplica,Não Assinado,2023-01-02 20:42:32.557,Sem Conexão
1864,50816,Não se Aplica,Não Assinado,2023-01-02 10:08:33.887,Sem Conexão
1425,49188,Não se Aplica,Válido,2023-01-05 17:34:15.690,OK
118,32481,Não se Aplica,Não Assinado,2023-01-15 12:30:50.247,Sem Conexão
40,29414,Não Assinado,Não Assinado,2023-01-10 18:02:51.707,Sem Conexão
1856,50808,Não se Aplica,Não Assinado,2023-01-02 10:33:57.933,Sem Conexão
237,34436,Não se Aplica,Válido,2023-01-05 18:32:56.877,OK
383,37401,Não se Aplica,Não Assinado,2023-01-04 14:12:16.077,Sem Conexão
1536,49472,Não se Aplica,Válido,2023-01-03 13:15:21.793,OK
1867,50840,Não se Aplica,Válido,2023-01-05 18:59:20.817,OK


,IdeUsinaOutorga,NomSituacaoContratoCUSD,NomSitContratoCUST,CondicaoConexao
245,34629,Não se Aplica,Não Assinado,Sem Conexão
115,32405,Não se Aplica,Válido,OK
1409,49171,Não se Aplica,Válido,OK
249,34656,Não se Aplica,Válido,OK
718,43154,Não se Aplica,Válido,OK
1315,48579,Não se Aplica,Não Assinado,Sem Conexão
892,45025,Válido,Não se Aplica,OK
1297,47489,Válido,Não se Aplica,Verificar
1794,50205,Não Assinado,Não se Aplica,Sem Conexão
2244,54623,Não se Aplica,Não Assinado,Sem Conexão


,IdeUsinaOutorga,DscComercializacaoEnergia,DatConclusaoACL,PPA
897,45063,Fora do ACR,NaT,Nenhum
2114,52383,Fora do ACR,NaT,Nenhum
1263,47455,Fora do ACR,NaT,Nenhum
160,33228,ACR,NaT,ACR
1695,49862,Fora do ACR,NaT,Nenhum
1930,51424,ACR,NaT,ACR
2021,51821,Fora do ACR,NaT,Nenhum
730,43168,Fora do ACR,NaT,Nenhum
1272,47464,Fora do ACR,NaT,Nenhum
573,38120,Fora do ACR,2021-09-14,ACL


In [371]:
ea = "Em andamento"
na = "Não Iniciada"
nenhum = "Nenhum"
paralisada = "Paralisada"
ok = "OK"

df_usina['criterio'] = np.select(
    [
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum))
    ],
    [0,1,2,3,4,5,8,7],
    default=6
)


df_usina['dscriterio'] = np.select(
    [
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum))
    ],
    [
    "Usina em Obras em andamento",
    "Sem obras e PPA Ok",
    "Sem obras, sem PPA e LI Ok e Cust OK",
    "Sem obras, sem PPA e LI OK e Cust não Ok",
    "Sem obras, sem PPA e LP OK e Cust Ok",
    "Sem obras, sem PPA e LP OK e Cust não Ok",
    "Obras paralisadas",
    "Sem obras, sem PPA e sem Rapeel"
],
    default="Sem LP")

display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio','dscriterio']].sample(10))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio,dscriterio
496,37814,Não Iniciada,ACR,LP,Sem Uso,1,Sem obras e PPA Ok
154,32937,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
1181,47341,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
924,45706,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2125,53474,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
2377,56421,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
2021,51821,Não Iniciada,Nenhum,LP,Sem Uso,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
205,34183,Não Iniciada,Nenhum,LP,Verificar,5,"Sem obras, sem PPA e LP OK e Cust não Ok"
593,38210,Não Iniciada,Nenhum,Não Informado,Não informado,7,"Sem obras, sem PPA e sem Rapeel"
845,44568,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"


In [372]:
slp = "Sem LP"
sim = "Sim"
df_usina['criterio_novo'] = np.select(
    [
        (df_usina.IdcSemPrevisao == sim),
        (df_usina.IdcSituacaoObra == ea),
        (( df_usina.IdcSituacaoObra == na)  &  (df_usina.PPA != nenhum) ),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso == ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental.isin(["LI","LO"])) & (df_usina.CondicaoUso != ok)),
        ((df_usina.IdcSituacaoObra == na) & (df_usina.PPA == nenhum) & (df_usina.CondicaoAmbiental == "LP") & (df_usina.CondicaoUso != ok)),
        (df_usina.IdcSituacaoObra == paralisada),
        (df_usina.CondicaoAmbiental == "Sem LP"),
        ((df_usina.DthEnvio.isna()) & (df_usina.PPA == nenhum)),
        
    ],
    [9,0,1,2,4,3,5,8,6,7],
    default=-8
)


display(df_usina[['IdeUsinaOutorga','IdcSituacaoObra','PPA','CondicaoAmbiental','CondicaoUso','criterio_novo','dscriterio']].sample(5))

,IdeUsinaOutorga,IdcSituacaoObra,PPA,CondicaoAmbiental,CondicaoUso,criterio_novo,dscriterio
1822,50525,Não Iniciada,Nenhum,LI,OK,2,"Sem obras, sem PPA e LI Ok e Cust OK"
1739,49961,Não Iniciada,Nenhum,LO,Verificar,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1227,47394,Não Iniciada,Nenhum,LO,Sem Uso,3,"Sem obras, sem PPA e LI OK e Cust não Ok"
1438,49201,Não Iniciada,Nenhum,Sem LP,OK,9,Sem LP
1216,47380,Não Iniciada,Nenhum,LP,OK,4,"Sem obras, sem PPA e LP OK e Cust Ok"


In [373]:
df_usina[df_usina.IdcSituacaoObra == paralisada][["IdeUsinaOutorga",'criterio_novo','IdcSemPrevisao','IdcSituacaoObra']]

,IdeUsinaOutorga,criterio_novo,IdcSemPrevisao,IdcSituacaoObra
2,659,9,Sim,Paralisada
9,27096,9,Sim,Paralisada
18,28000,9,Sim,Paralisada
19,28001,9,Sim,Paralisada
25,28492,9,Sim,Paralisada
54,30150,9,Sim,Paralisada
77,31271,9,Sim,Paralisada
83,31535,9,Sim,Paralisada
84,31536,9,Sim,Paralisada
85,31562,9,Sim,Paralisada


# Informações de UGs

## Carrega banco de dados

In [374]:
cols = ["DatUGInicioOpComerOutorgado","IdcMonitorada","IdeUsinaOutorga",'NumUgUsina',"DatLiberacaoSFGTeste","DatInicioOpTesteOutorgada",'DatLiberOpComerRealizado']
monitoramentoug = pd.read_parquet(os.path.join(directory,"vmonitoramentoug.gzip"))[cols].rename(columns={'NumUgUsina':'NumOperacaoUg'})

In [375]:
# Carrega BD vrapeeloperacaoug => ug_rapeel
cols = ["IdeUsinaOutorga",'NumOperacaoUg',"DthEnvio",'DatPrevistaComercial']
ug_rapeel = pd.read_parquet(os.path.join(directory,"vrapeeloperacaoug.gzip"))[cols]
ug_rapeel = ug_rapeel.loc[ug_rapeel.groupby(['IdeUsinaOutorga','NumOperacaoUg']).DthEnvio.idxmax()]

In [376]:
df_ug = pd.merge(df_usina,monitoramentoug,on="IdeUsinaOutorga",how='left')
df_ug = pd.merge(df_ug,ug_rapeel,on=list_id_ug,how='left')
df_ug = df_ug.loc[df_ug.DatLiberOpComerRealizado.isna()].reset_index(drop=True)

In [377]:
previsao_path = os.path.join(get_standard_folder_path("Documents"),r"Previsor\Previsoes")
previsao_file = sorted(glob.glob(f"{previsao_path}/*.gzip"))[-1]
calculo_previsao = pd.read_parquet(previsao_file)
calculo_previsao.FaseAtual =calculo_previsao.FaseAtual.str.slice(0,-3)
calculo_previsao.Indicador =calculo_previsao.Indicador/100

In [378]:
cols = ["Dat_OC_obrigacao","DatPrevisaoIniciobra","FASE","Ind_crono_norm","IdeUsinaOutorga","flagOPTeste30dias",'Previsao_OC']
rename_cols = {'FaseAtual':'FASE','Indicador':'Ind_crono_norm','NumUgUsina':'NumOperacaoUg'}
calculo_previsao = calculo_previsao.rename(columns=rename_cols)

for col in ['SigTipoGeracao','DatMonitoramento','DatInicioObraOutorgado','DatPrevisaoIniciobra']:
    if col in calculo_previsao.columns:
        calculo_previsao.drop([col],inplace=True,axis=1)

In [379]:
df_ug = pd.merge(df_ug,calculo_previsao,on=list_id_ug,how="left")

##  Cria colunas com definições e regras

In [380]:
################################ regra_antiga ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'regra_antiga'] = pd.NA

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'regra_antiga'] = df_ug[['Previsao_OC','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regra_antiga'] = ( hoje +pd.Timedelta(60,'D'))

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'regra_antiga'] = df_ug['Previsao_OC']

df_ug.loc[( (df_ug['criterio'] == 1) | (df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Sim")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[( (df_ug['criterio'] == 1) | (df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

df_ug.loc[(df_ug['criterio'] >= 3) & ( (df_ug['IdcObraParalisada'] == "Sim")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[df_ug['criterio'] >= 3 & ( (df_ug['IdcObraParalisada'] == "Não")),'regra_antiga'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)


################################ regrapmo ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'regrapmo'] =  pd.NA

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'regrapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regrapmo'] = ( hoje + pd.Timedelta(60,'D'))

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'regrapmo'] = df_ug['Previsao_OC']

df_ug.loc[( (df_ug['criterio'] == 8) ),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)

df_ug.loc[( (df_ug['criterio'] == 1) | (df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC']].max(axis=1)

df_ug["Dummy"] = hoje + pd.Timedelta(3*365,'D')

df_ug.loc[(df_ug['criterio'] == 3),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

df_ug["Dummy"] = hoje + pd.Timedelta(4*365,'D')
df_ug.loc[(df_ug['criterio'] == 4),'regrapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')
df_ug.loc[(df_ug['criterio'] == 5) | (df_ug['criterio'] == 7),'regrapmo'] =  df_ug[['DatPrevistaComercial','Dat_OC_obrigacao','Previsao_OC','Dummy']].max(axis=1)

df_ug.loc[(df_ug['criterio'] == 6),'regrapmo'] = pd.NA


################################ criterionovopmo ################################
list_condicoes = [
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 8)),
    ((df_ug['criterio_novo'] == 9)),
]

list_values = [0.1, 0.2, 0.3, 1, 2, 3, 5, 4, 6, 7, 8, 9]

df_ug['criterionovopmo'] = np.select(list_condicoes,list_values)

list_condicoes = [
    ((df_ug['criterio_novo'] == 9)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT")),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1)),
    ((df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT")),
    ((df_ug['criterio_novo'] == 1)),
    ((df_ug['criterio_novo'] == 2)),
    ((df_ug['criterio_novo'] == 3)),
    ((df_ug['criterio_novo'] == 4)),
    ((df_ug['criterio_novo'] == 5)),
    ((df_ug['criterio_novo'] == 7)),
    ((df_ug['criterio_novo'] == 6)),
    ((df_ug['criterio_novo'] == 8))
]


list_values = [
    "Usina Viabilidade Baixa análise da fiscalização",
    "Usina em obras = Previsão OC maior entre data calculada e data RAPEEL",
    "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias",
    "Usina em Teste = Previsão OC conforme data calculada",
    "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso",
    "Usina sem obras, sem cust, sem PPA e com LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, com cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem cust, sem PPA e sem LI  = Previsão OC handicap de 5 anos",
    "Usina sem obras, sem PPA e sem RAPEEL  = Previsão OC handicap de 5 anos",
    "Usina sem LP = Previsão OC sem previsão","Usina obras paralisadas = Previsão OC maior entre data RAPEEL e data compromisso"
]
df_ug['dsccriterionovo'] = np.select(list_condicoes,list_values)


################################ regranovapmo ################################
df_ug.loc[(df_ug['criterio_novo'] == 9),'regranovapmo'] = pd.NA

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] != "OT"),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC']].max(axis=1)

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'regranovapmo'] = ( hoje + pd.Timedelta(60,"D"))

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].isna()),'regranovapmo'] = df_ug['DatPrevistaComercial']

df_ug.loc[(df_ug['criterio_novo'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['Previsao_OC'].notna()),'regranovapmo'] = df_ug['Previsao_OC']

df_ug.loc[(df_ug['criterio_novo']== 1) | (df_ug['criterio_novo']== 2) ,'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao']].max(axis=1)
df_ug["Dummy"] = hoje + pd.Timedelta(5*365,'D')

df_ug.loc[((df_ug['criterio_novo'] == 3) | (df_ug['criterio_novo'] == 4) | (df_ug['criterio_novo'] == 5) ),'regranovapmo'] = df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

df_ug.loc[df_ug['criterio_novo']== 6,'regranovapmo'] = pd.NA

df_ug["Dummy"] = hoje + pd.Timedelta(6*365,'D')
df_ug.loc[df_ug['criterio_novo']== 7,'regranovapmo'] =  df_ug[['DatPrevistaComercial','Previsao_OC','Dat_OC_obrigacao','Dummy']].max(axis=1)

df_ug.loc[df_ug['criterio_novo']== 8,'regranovapmo'] = df_ug[['DatPrevistaComercial','Dat_OC_obrigacao']].max(axis=1)


################################ criterio_pmo_ts ################################
list_condicoes = [
    (df_ug['IdcSemPrevisao'] == "Sim"),
    (df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),
    (df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),
    (df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),
    ((df_ug['PPA'] != "Nenhum")) & ((df_ug['criterio'] == 8) ),
    ( (df_ug['criterio'] == 1)) & ( (df_ug['IdcObraParalisada'] == "Não")),
    ((df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),
    ( (df_ug['criterio'] == 8)) & ( (df_ug['PPA'] == "Nenhum") & (df_ug['CondicaoUso'] == "OK")) & ( (df_ug['CondicaoAmbiental'] == "LI") | (df_ug['CondicaoAmbiental'] == "LO") ),
    (df_ug['criterio'] == 3),
    (df_ug['criterio'] == 4),
    (df_ug['criterio'] == 5),
    (df_ug['criterio'] == 7),
    (df_ug['criterio'] == 6),
]

list_values = [8,0.1,0.2,0.3,1.1,1.2,2.2,2.1,3,4,5,7,6]
df_ug['criterio_pmo_ts'] = np.select(list_condicoes,list_values)


################################ dsccriterio_pmo_ts ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias == Previsão OC próximos 60 dias"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste == Previsão OC conforme data calculada"

df_ug.loc[((df_ug['PPA'] != "Nenhum")) & ((df_ug['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

df_ug.loc[( (df_ug['criterio'] == 1)) & ( (df_ug['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'dsccriterio_pmo_ts'] = "Usina Viabilidade Baixa análise da fiscalização"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] != "OT"),'dsccriterio_pmo_ts'] = "Usina em obras == Previsão OC maior entre data calculada e data RAPEEL"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT") & (df_ug['flagOPTeste30dias'] == 1),'dsccriterio_pmo_ts'] = "Usina em Teste há mais de 30 dias = Previsão OC próximos 60 dias"

df_ug.loc[(df_ug['criterio'] == 0) & (df_ug['FASE'] == "OT"),'dsccriterio_pmo_ts'] = "Usina em Teste = Previsão OC conforme data calculada"

df_ug.loc[((df_ug['PPA'] != "Nenhum")) & ((df_ug['criterio'] == 8) ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas com PPA = Previsão OC maior entre data RAPEEL e data compromisso"

df_ug.loc[( (df_ug['criterio'] == 1)) & ( (df_ug['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras com PPA = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[((df_ug['criterio'] == 2)) & ( (df_ug['IdcObraParalisada'] == "Não")),'dsccriterio_pmo_ts'] = "Usina sem obras sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[( (df_ug['criterio'] == 8)) & ( (df_ug['PPA'] == "Nenhum") & (df_ug['CondicaoUso'] == "OK")) & ( (df_ug['CondicaoAmbiental'] == "LI") | (df_ug['CondicaoAmbiental'] == "LO") ),'dsccriterio_pmo_ts'] = "Usina obras paralisadas sem PPA com LI com CUST = Previsão OC maior entre data calculada, data RAPEEL e data compromisso"

df_ug.loc[(df_ug['criterio'] == 3),'dsccriterio_pmo_ts'] = "Usina sem PPA com LI sem CUST = Previsão OC maior entre handicap de 3 anos e data RAPEEL e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 4),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI com CUST = Previsão OC maior entre handicap de 4 anos e data RAPEEL e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 5),'dsccriterio_pmo_ts'] = "Usina sem PPA sem LI sem CUST = Previsão OC maior entre handicap de 5 anos e data RAPEEL e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 7),'dsccriterio_pmo_ts'] = "Usina sem obras, sem PPA e sem RAPEEL = Previsão OC maior entre handicap de 5 anos e data compromisso e data calculada"

df_ug.loc[(df_ug['criterio'] == 6),'dsccriterio_pmo_ts'] = "Usina sem LP = Previsão OC sem previsão"


################################ criterio_pmo_ts_max_usina ################################
criterio_pmo_ts_max = df_ug.groupby("IdeUsinaOutorga")[['criterio_pmo_ts']].max().rename(columns=
{'criterio_pmo_ts': 'criterio_pmo_ts_max_usina'})
df_ug = df_ug.merge(criterio_pmo_ts_max,on="IdeUsinaOutorga",how='left')


################################ dscjustificativaregra ################################
df_ug.loc[(df_ug['IdcSemPrevisao'] == "Sim"),'dscjustificativaregra'] = "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina"

df_ug.loc[(df_ug['IdcObraParalisada'] == "Sim") & (df_ug['IdcSemPrevisao'] != "Sim"),'dscjustificativaregra'] = "Paralisação de obras"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 0.2),'dscjustificativaregra'] = "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 0.1) | (df_ug['criterio_pmo_ts'] == 0.3),'dscjustificativaregra'] = "Estágio das obras da usina"


df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 1.1),'dscjustificativaregra'] = "Paralisação de obras"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 1.2),'dscjustificativaregra'] = "Compromisso de venda de energia - PPA"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 2.2),'dscjustificativaregra'] = "Licença de Instalação - LI e acesso a rede - CUST/CUSD"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 3),'dscjustificativaregra'] = "Acesso a rede contratado - CUST/CUSD"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 4),'dscjustificativaregra'] = "Acesso a rede não contratado - CUST/CUSD"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 5),'dscjustificativaregra'] = "Sem Licença de Instalação - LI"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 6),'dscjustificativaregra'] = "Nenhuma licença ambiental válida"

df_ug.loc[(df_ug.criterio_pmo_ts_max_usina== 7),'dscjustificativaregra'] = "Sem RAPEEL"


################################ dscjustificativaregranova ################################
list_condicoes = [
    (df_ug['criterionovopmo'] == 9),
    (df_ug['criterionovopmo'] == 8),
    (df_ug['criterionovopmo'] == 0.2),
    (df_ug['criterionovopmo'] == 0.1) | (df_ug['criterio_pmo_ts'] == 0.3),
    (df_ug['criterionovopmo'] == 1),
    df_ug['criterionovopmo']==2,
    df_ug['criterionovopmo']==3,
    (df_ug.criterio_pmo_ts_max_usina== 4),
    (df_ug.criterio_pmo_ts_max_usina== 5),
    (df_ug.criterio_pmo_ts_max_usina== 6),
    (df_ug.criterio_pmo_ts_max_usina== 7),
]

list_values = [
    "Analisar justificativa: Revogação da outorga em avaliação ou Demandas judiciais ou Inviabilidade da implantação da usina",
    "Paralisação de obras",
    "Analisar justificativa: Sem Licença de Operação (LO) ou Alterações de características técnicas ou Estágio das obras da conexão associada ou Demandas judiciais ou Paralisação de obras",
    "Estágio das obras da usina",
    "Compromisso de venda de energia - PPA",
    "Acesso a rede contratado - CUST/CUSD",
    "Acesso a rede não contratado - CUST/CUSD",
    "Sem Licença de Instalação - LI",
    "Sem Licença de Instalação - LI",
    "Nenhuma licença ambiental válida",
    "Sem RAPEEL"
]

df_ug['dscjustificativaregranova'] = np.select(list_condicoes,list_values)


In [381]:
list_criterios = ["criterio",'criterio_novo',"criterionovopmo",'criterio_pmo_ts','criterio_pmo_ts_max_usina']
list_justificativas = ['dsccriterionovo','dsccriterio_pmo_ts','dscjustificativaregranova']

# Testes

## Junta dados do SAS e Python

In [382]:
criterios_da_previsao_sas = load_pickle('./criterios_da_previsao_sas.pickle')
tabela_criterio_sas = load_pickle('./tabela_criterio_sas.pickle')
criterios_da_previsao_sas = pd.merge(criterios_da_previsao_sas,tabela_criterio_sas,on="IdeUsinaOutorga").rename(columns={'NumUgUsina':'NumOperacaoUg'})

criterios_da_previsao_sas['SAS'] = True
criterios_da_previsao_sas.CalculoPrevisorOC_mx = criterios_da_previsao_sas.CalculoPrevisorOC_mx.dt.normalize()
df_ug["PY"] = True

In [383]:
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaPMO = criterios_da_previsao_sas.DscJustificativaPMO.str.replace('_x002F_','/')

criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscJustificativaRegranova = criterios_da_previsao_sas.DscJustificativaRegranova.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0020_',' ')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002F_','/')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0028_','(')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x0029_',')')

criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x003D_','=')
criterios_da_previsao_sas.DscCriterioNovo = criterios_da_previsao_sas.DscCriterioNovo.str.replace('_x002C_', ',')

In [384]:
merge = pd.merge(df_ug,criterios_da_previsao_sas,on=['IdeUsinaOutorga','NumOperacaoUg'],suffixes=('_py','_sas'),how='outer')
merge['OC_diff'] = merge.Previsao_OC - merge.CalculoPrevisorOC_mx

## Teste previsão

In [385]:
merge[['IdeUsinaOutorga','NumOperacaoUg'] + [ 'Previsao_OC','CalculoPrevisorOC_mx','OC_diff','IdcUsinaMonitorada']].sort_values(by='OC_diff',ascending=False).head(10)

,IdeUsinaOutorga,NumOperacaoUg,Previsao_OC,CalculoPrevisorOC_mx,OC_diff,IdcUsinaMonitorada
119,30114,1,2024-08-05,2021-12-01,978 days,Sim
138,30414,1,2025-03-18,2022-09-08,922 days,Sim
120,30114,2,2024-08-19,2022-05-19,823 days,Sim
144,30612,1,2025-03-29,2023-01-23,796 days,Sim
118,30114,3,2024-08-25,2022-08-11,745 days,Sim
11619,37731,1,2024-09-18,2023-02-24,572 days,Sim
11618,37731,3,2024-09-18,2023-02-24,572 days,Sim
11617,37731,2,2024-09-18,2023-02-24,572 days,Sim
11613,37730,1,2024-09-23,2023-05-16,496 days,Sim
11615,37730,2,2024-09-23,2023-05-16,496 days,Sim


## Teste critérios

### FASE ok

In [386]:
merge[merge.FASE_py != merge.FASE_sas].shape

(0, 207)

### criterionovopmo OK

In [387]:
merge[merge.CriterioNovoPMO != merge.criterionovopmo][['IdeUsinaOutorga','NumOperacaoUg'] + ['IdcSemPrevisao','flagOPTeste30dias_sas','flagOPTeste30dias_py','FASE_sas','FASE_py','criterio_novo','criterio_sas','criterio_py','CriterioNovoPMO','criterionovopmo']]#.drop_duplicates(subset='IdeUsinaOutorga')

,IdeUsinaOutorga,NumOperacaoUg,IdcSemPrevisao,flagOPTeste30dias_sas,flagOPTeste30dias_py,FASE_sas,FASE_py,criterio_novo,criterio_sas,criterio_py,CriterioNovoPMO,criterionovopmo


### criterio OK

In [388]:
merge[merge.criterio_sas != merge.criterio_py].shape

(0, 207)

### dscjustificativaregranova OK

In [389]:
merge[merge.dscjustificativaregranova != merge.DscJustificativaRegranova][['dscjustificativaregranova','DscJustificativaRegranova']]


,dscjustificativaregranova,DscJustificativaRegranova


### dsccriterionovo OK

In [392]:
merge[merge.dsccriterionovo != merge.DscCriterioNovo]

,IdeUsinaOutorga,DatCanteiroObraRealizado,DatDesvioRioRealizado,DatEnchimentoRealizado,DatConclusaoSisTransRealizado,DatPrevisaoIniciobra_py,IdcObraParalisada,IdcUsinaMonitorada,DatMontagemOutorgado,DatConcretagemRealizado,DatConclusaoTorresRealizado,DatInicioObraOutorgado_py,DatConcretagemOutorgado,DatSisTransmissaoRealizado,IdcSituacaoObra,IdcSemPrevisao,DatComissionamentoUGRealizado,DatMonitoramento_py,SigTipoGeracao,DscComercializacaoEnergia_py,DatInicioObraRealizado,DatMontagemRealizado,CodLeilao,DatInicioSuprimento,CodCeg,DthEnvio_x_py,canteiroReal,prev_IO,IO_real,DesvRio_real,Ench_Real,IOTrans_Real,Conc_Trans_REAL,Comiss_Real,ME_Real_conc_eol,Conc_Combust_Real,DatRealizacaoVI,DatRealizacaoX,DatRealizacaoV,DatRealizacaoIV,DatConclusaoACL_py,DatValidadeLI_py,DatValidadeLP_py,DatValidadeLO_py,NomSitContratoCCD_py,NomSitContratoCUST_py,NomSituacaoContratoCUSD_py,NomSitContratoCCT_py,ValidadeAmbiental,CondicaoAmbiental,...,DatValidadeContratoCCD,NomSitContratoCCT_sas,DatAssinaturaContratoCCT,DatValidadeContratoCCT,NomSituacaoContratoCUSD_sas,DatAssinaturaContratoCUSD,DatValidadeContratoCUSD,NomSitContratoCUST_sas,DatAssinaturaContratoCUST,DatValidadeContratoCUST,DatSolicitacaoLP,DatPrevistaEmissaoLP,DatEmissaoLP,DatValidadeLP_sas,NomSituacaoLP,DatSolicitacaoLI,DatPrevistaEmissaoLI,DatEmissaoLI,DatValidadeLI_sas,NomSituacaoLI,DatSolicitacaoLO,DatPrevistaEmissaoLO,DatEmissaoLO,DatValidadeLO_sas,NomSituacaoLO,IdcTipoEntrave_y,DscSolucao_y,DscObservacaoAcesso_y,DscObservacaomercado_y,DscObservacaoAmbiental_y,ValidadeAmbiental_y,CondicaoAmbiental_y,CondicaoConexao_sas,CondicaoUso_y,PPA_y,DscComercializacaoEnergia_sas,DscSistema_y,IdcViabilidade_y,IdcSituacaoObra_y,IdcObraParalisada_y,IdcSemPrevisão_y,criterio_sas,DatPrevistaComercialRapeelMX_y,DscJustificativaPrevisao,DscObservacaoUG_y,DatEmissaoAto_y,criterio_x0020_novo,dscriterio_sas,SAS,OC_diff


# BIU

In [ ]:
df_ug["classe"] = df_ug.CodCeg.str.slice(3,5)
df_ug["CC_real"] = pd.to_datetime(pd.NA)
df_ug.loc[df_ug.classe == "PH","CC_real"] = df_ug.DatRealizacaoIV
df_ug.loc[df_ug.classe == "CV","CC_real"] = df_ug.DatRealizacaoV
df_ug["ME_real"] = df_ug.DatRealizacaoX
df_ug.loc[df_ug.classe == "CV","ME_real"] = df_ug.DatRealizacaoVI

In [ ]:
ug_com_previsao = df_ug.loc[df_ug.DatPrevistaComercial.notna()][list_id_ug + ['DatPrevistaComercial']]
ug_com_previsao = ug_com_previsao.loc[ug_com_previsao.groupby('IdeUsinaOutorga').DatPrevistaComercial.idxmax()][['IdeUsinaOutorga','DatPrevistaComercial']].rename(
    columns = {'DatPrevistaComercial': 'DatPrevistaComercial_MaxUsina'})
df_ug = pd.merge(df_ug,ug_com_previsao,on='IdeUsinaOutorga')